# Check Number of FEL trains (single bunch) in TimePix HDF5 files

In [1]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


#### Imports

In [2]:
import numpy as np  
import matplotlib.pyplot as plt
import h5py
import os

#### Utility functions

In [7]:
def generate_hdf_file_name(run_number):
    try:
        #hdf_file_path = '/asap3/flash/gpfs/bl1/2019/data/11006902/processed/timepix_hdf/'
        hdf_file_path = '/gpfs/bl1/current/processed/timepix_hdf/'
        file_start = "run_"+str(run_number).zfill(4)
        not_file_end = 'rawOnly.hdf5'
        hdf_file = [i for i in os.listdir(hdf_file_path) if os.path.isfile(os.path.join(hdf_file_path,i)) and i.startswith(file_start) and not i.endswith(not_file_end)][0]
        hdf_file_complete_path = hdf_file_path+hdf_file
        assert os.path.isfile(hdf_file_complete_path), 'File does not exist!'
        return hdf_file_complete_path
    except IndexError:
        print("Run",run_number,"do not exist!")

def get_train_numbers_from_file(run_interval):
    number_of_trains = []
    runs = []
    for i in range(run_interval[0], run_interval[1]+1):
        runs.append(i)
        hdf_file = generate_hdf_file_name(i)
        if hdf_file != None:
            number_of_trains.append(number_of_trains_from_hdf(hdf_file))
        elif hdf_file == None:
            number_of_trains.append(0)
    return runs, number_of_trains

def number_of_trains_from_hdf(hdf_file_complete_path):
    '''Retrun number of recorded FEL trains in HDF file'''
    with h5py.File(hdf_file_complete_path, 'r') as h_file:
        trains = len(h_file['tpx3Times/triggerNr'][:])
    return trains

def unique(list_to_check):  
    unique_list = [] 
    for x in list_to_check: 
        if x not in unique_list: 
            unique_list.append(x) 
    return unique_list

def get_trains_vs_unique_delay(delay_pos, trains):
    unique_delay_pos = unique(delay_pos)
    unique_trains = []
    for i in range(len(unique_delay_pos)):
        tt = []
        for j in range(len(delay_pos)):
            if unique_delay_pos[i] == delay_pos[j]:
                tt.append(trains[j])
        unique_trains.append(np.sum(np.array(tt)))
    sort_ind = np.argsort(np.array(unique_delay_pos))
    unique_delay_pos = np.array(unique_delay_pos)[sort_ind]
    unique_trains = np.array(unique_trains)[sort_ind]
    return unique_delay_pos, unique_trains

def plot_number_of_trains_vs_run(run_interval): 
    runs, trains = get_train_numbers_from_file(run_interval)              
    fig = plt.figure(figsize = (10,10))
    plt.plot(runs,trains,'o')
    plt.ylabel('FEL pulses')
    plt.xlabel('Run Number')
    plt.show()
        
def list_number_of_trains_vs_run(run_interval): 
    runs, trains = get_train_numbers_from_file(run_interval)
    for i in range(len(runs)):
        print('RunNumber:',runs[i],'| number of trains:',trains[i])

def plot_number_of_trains_vs_delay(run_interval):
    runs, trains = get_train_numbers_from_file(run_interval)
    runs_pp, delay_pos = get_delay_stage_pos_from_txt_file(run_interval)
    assert runs == runs_pp, 'Runs are not identical!'
    unique_delay_pos, unique_trains = get_trains_vs_unique_delay(delay_pos, trains)
    fig = plt.figure(figsize = (10,10))
    plt.plot(unique_delay_pos,unique_trains,'o')
    plt.ylabel('FEL pulses')
    plt.xlabel('Delay Position')
    plt.show()
        
def list_number_of_trains_vs_delay(run_interval):
    runs, trains = get_train_numbers_from_file(run_interval)
    runs_pp, delay_pos = get_delay_stage_pos_from_txt_file(run_interval)
    assert runs == runs_pp, 'Runs are not identical!'
    unique_delay_pos, unique_trains = get_trains_vs_unique_delay(delay_pos, trains)
    for i in range(len(unique_delay_pos)):
        print('Delay Position:', unique_delay_pos[i],'| number of trains:',unique_trains[i])           

def get_delay_stage_pos_from_txt_file(run_interval):
    runs = []
    delay_pos = []
    filepath = '/home/bl1user/Desktop/erk20919/CAMP/beamtime/run_pp-delay.txt'
    with open(filepath) as fp:
        for cnt, line in enumerate(fp):
            tt = [x.strip() for x in line.split(',')]
            runs.append(int(tt[0]))
            delay_pos.append(float(tt[1]))
    delay_pos = delay_pos[runs.index(run_interval[0]):runs.index(run_interval[1])+1]
    runs = runs[runs.index(run_interval[0]):runs.index(run_interval[1])+1]
    return runs, delay_pos


### Interval of runs

In [34]:
#run_interval = [429, 521] 
run_interval = [526, 570] 

### Number of trains in runs

In [35]:
# plot_number_of_trains_vs_run(run_interval)
list_number_of_trains_vs_run(run_interval)

Run 563 do not exist!
Run 564 do not exist!
Run 565 do not exist!
Run 566 do not exist!
Run 567 do not exist!
Run 568 do not exist!
Run 569 do not exist!
Run 570 do not exist!
RunNumber: 526 | number of trains: 2619
RunNumber: 527 | number of trains: 2858
RunNumber: 528 | number of trains: 2801
RunNumber: 529 | number of trains: 2744
RunNumber: 530 | number of trains: 2805
RunNumber: 531 | number of trains: 2810
RunNumber: 532 | number of trains: 2804
RunNumber: 533 | number of trains: 2844
RunNumber: 534 | number of trains: 2831
RunNumber: 535 | number of trains: 2842
RunNumber: 536 | number of trains: 2808
RunNumber: 537 | number of trains: 2748
RunNumber: 538 | number of trains: 2876
RunNumber: 539 | number of trains: 2801
RunNumber: 540 | number of trains: 2839
RunNumber: 541 | number of trains: 2851
RunNumber: 542 | number of trains: 2838
RunNumber: 543 | number of trains: 2845
RunNumber: 544 | number of trains: 2867
RunNumber: 545 | number of trains: 2807
RunNumber: 546 | number 

### Number of trains in delay steps

In [ ]:
# plot_number_of_trains_vs_delay(run_interval)
list_number_of_trains_vs_delay(run_interval)